In [59]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
#from bokeh.plotting import figure, show, output_file


data1 = pd.read_csv(r'细胞间相似性（皮尔森）.csv')

In [60]:
# 细胞系-细胞系
high_data = data1.set_index(data1.loc[:,"Unnamed: 0"]).drop(["Unnamed: 0"], axis=1)
high_data

,A2058_SKIN,A2780_OVARY,A375_SKIN,A427_LUNG,CAOV3_OVARY,COLO320_LARGE_INTESTINE,EFM192A_BREAST,ES2_OVARY,HCT116_LARGE_INTESTINE,HT144_SKIN,...,SKMES1_LUNG,SKOV3_OVARY,SW620_LARGE_INTESTINE,SW837_LARGE_INTESTINE,T47D_BREAST,UACC62_SKIN,VCAP_PROSTATE,ZR751_BREAST,DLD1_LARGE_INTESTINE,PA1_OVARY
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A2058_SKIN,1.000000,0.933131,0.895933,0.872833,0.930594,0.913048,0.828992,0.910626,0.885239,0.939526,...,0.891651,0.853359,0.925756,0.903832,0.879188,0.944257,0.908624,0.859199,0.831640,0.809304
A2780_OVARY,0.933131,1.000000,0.832244,0.911416,0.931201,0.964174,0.863401,0.868830,0.885013,0.878791,...,0.884535,0.830163,0.931660,0.927448,0.905887,0.881719,0.947997,0.847023,0.778585,0.780041
A375_SKIN,0.895933,0.832244,1.000000,0.760185,0.841720,0.823737,0.665576,0.793831,0.843605,0.929210,...,0.732954,0.815884,0.869311,0.792255,0.723077,0.878075,0.840062,0.868717,0.816548,0.824151
A427_LUNG,0.872833,0.911416,0.760185,1.000000,0.893758,0.891928,0.818783,0.788311,0.825792,0.818094,...,0.818817,0.809946,0.844469,0.868943,0.914153,0.866080,0.873004,0.764425,0.756926,0.665792
CAOV3_OVARY,0.930594,0.931201,0.841720,0.893758,1.000000,0.914899,0.828365,0.854038,0.892419,0.888962,...,0.897316,0.887472,0.919940,0.930485,0.863734,0.899954,0.889967,0.852847,0.813600,0.774179
COLO320_LARGE_INTESTINE,0.913048,0.964174,0.823737,0.891928,0.914899,1.000000,0.854525,0.872164,0.880580,0.857782,...,0.879370,0.801244,0.935556,0.936820,0.884787,0.880339,0.937084,0.882383,0.777515,0.779708
EFM192A_BREAST,0.828992,0.863401,0.665576,0.818783,0.828365,0.854525,1.000000,0.798369,0.761709,0.763698,...,0.855413,0.717744,0.840727,0.862328,0.881492,0.798973,0.833114,0.752829,0.693466,0.641782
ES2_OVARY,0.910626,0.868830,0.793831,0.788311,0.854038,0.872164,0.798369,1.000000,0.813371,0.845340,...,0.884428,0.779564,0.892501,0.857552,0.819010,0.853913,0.853222,0.816232,0.774859,0.749871
HCT116_LARGE_INTESTINE,0.885239,0.885013,0.843605,0.825792,0.892419,0.880580,0.761709,0.813371,1.000000,0.836212,...,0.785440,0.840068,0.897913,0.888780,0.780899,0.843875,0.842408,0.832398,0.787495,0.814974


In [61]:
# 药物对-细胞系
data2 = pd.read_csv('label_gai.csv')#(r'Label_synergy4000.xlsx')
data2

,Unnamed: 0,A2058_SKIN,A2780_OVARY,A375_SKIN,A427_LUNG,CAOV3_OVARY,COLO320_LARGE_INTESTINE,EFM192A_BREAST,ES2_OVARY,HCT116_LARGE_INTESTINE,...,SKMES1_LUNG,SKOV3_OVARY,SW620_LARGE_INTESTINE,SW837_LARGE_INTESTINE,T47D_BREAST,UACC62_SKIN,VCAP_PROSTATE,ZR751_BREAST,DLD1_LARGE_INTESTINE,PA1_OVARY
0,0,-9.51,2.60,15.20,6.22,-16.4000,5.360,-7.080,8.050,12.400,...,-6.230,13.50,5.10,5.15,6.63,3.880,-1.48,-8.660,13.00,4.83
1,1,-13.20,-4.03,11.10,10.50,-15.6000,16.300,-1.720,8.890,0.686,...,2.820,1.51,13.80,-5.77,-14.40,-10.700,-16.80,-11.800,15.30,-13.10
2,2,26.40,14.50,29.50,17.50,17.7000,10.300,0.830,5.770,19.300,...,-12.200,-7.40,10.50,-4.97,13.10,18.800,12.00,5.950,6.77,18.90
3,3,4.33,-8.16,-5.42,-7.75,-14.3000,4.810,0.312,3.410,2.920,...,-3.140,-13.90,-4.57,-19.60,-2.72,3.680,14.40,-13.600,-3.64,-5.73
4,4,45.50,22.50,22.80,18.00,20.1000,22.700,6.300,24.900,35.400,...,24.400,31.20,18.40,12.90,14.30,34.700,21.20,14.400,26.40,55.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,578,1.35,11.70,20.20,9.17,46.6000,0.586,-37.700,-2.100,34.100,...,-0.217,-10.90,22.00,-2.43,-43.90,9.340,-4.83,-15.400,-4.42,17.80
579,579,9.19,8.01,7.50,-4.18,-6.7800,3.210,7.240,5.340,9.690,...,11.100,7.12,-2.42,6.96,14.90,9.120,10.60,14.700,11.60,14.20
580,580,15.40,12.40,19.20,-5.01,37.4000,8.590,9.770,3.770,10.600,...,4.330,5.84,-1.58,18.60,-20.70,25.100,20.30,3.750,-4.29,-1.44
581,581,-6.30,4.79,-23.30,-2.70,0.0856,-17.300,19.200,17.400,1.590,...,-13.900,-2.42,-2.61,19.10,-11.10,-13.500,9.05,-0.306,9.48,-4.29


In [62]:
columns = list(data2.columns[1:])
columns

['A2058_SKIN ',
 'A2780_OVARY ',
 'A375_SKIN ',
 'A427_LUNG ',
 'CAOV3_OVARY ',
 'COLO320_LARGE_INTESTINE ',
 'EFM192A_BREAST ',
 'ES2_OVARY ',
 'HCT116_LARGE_INTESTINE ',
 'HT144_SKIN ',
 'HT29_LARGE_INTESTINE ',
 'KPL1_BREAST ',
 'LOVO_LARGE_INTESTINE ',
 'MDAMB436_BREAST ',
 'MSTO211H_PLEURA ',
 'NCIH1650_LUNG ',
 'NCIH2122_LUNG ',
 'NCIH23_LUNG ',
 'NCIH460_LUNG ',
 'NCIH520_LUNG ',
 'OV90_OVARY ',
 'OVCAR4_OVARY ',
 'RKO_LARGE_INTESTINE ',
 'RPMI7951_SKIN ',
 'SKMEL30_SKIN ',
 'SKMES1_LUNG ',
 'SKOV3_OVARY ',
 'SW620_LARGE_INTESTINE ',
 'SW837_LARGE_INTESTINE ',
 'T47D_BREAST ',
 'UACC62_SKIN ',
 'VCAP_PROSTATE ',
 'ZR751_BREAST ',
 'DLD1_LARGE_INTESTINE ',
 'PA1_OVARY ']

In [63]:
# 删去对应行后的字典
dict_high_lines = {}
count = 0
for i in high_data.index:
    y = list(high_data.loc[:,i])
    y.pop(count)
    count += 1  
    dict_high_lines[i] = y

In [64]:
# 分离x y数据构造字典
dict_columns = {}
for i in columns: 
    
    # b
    y = data2.loc[:,i]
    
    # A
    temp = []
    for j in columns:
        if j != i:
            temp.append(j)
    m = data2.loc[:, temp[0]]
    tp = [m]
    for j in temp[1:]:
        tp.append(data2.loc[:, j])
    x = pd.DataFrame(tp).T
    
    # 找每列x相关度较大的5行
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    max_l = df.index[-13:]
    tp = []
    for j in max_l:
        tp.append(data2.loc[:, j])
    max_5x = pd.DataFrame(tp).T
    
    # 存储
    data = {
        "x": x,
        "y": y,
        "max_relevance": max_l,
        "max_5x": max_5x
    }
    dict_columns[i] = data

In [65]:
# 删去对应行后的字典
dict_high_lines = {}
count = 0
for i in high_data.index:
    y = list(high_data.loc[:,i])
    y.pop(count)
    count += 1
    dict_high_lines[i] = y
    # 找每列x相关度较大的5行
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    
    max_l = df.index[-13:] #取最大5个
    tp = []
    for j in max_l:
        tp.append(high_data.loc[j, :])
    max_5y = list(pd.DataFrame(tp).T.loc[[i]].values)[0]
    
    data = {
        "max_relevance": max_l,
        "y": y,
        "high_5": max_5y
    }
    dict_high_lines[i] = data

In [66]:
end_score = []
name_dict_high_lines = []
for elem in dict_high_lines:
    end_score.append(dict_high_lines[elem]['high_5'].mean())
    name_dict_high_lines.append(elem)
dict_high_lines_temp = {}
for score in end_score:
    if score >= 0.92:
        dict_high_lines_temp[name_dict_high_lines[end_score.index(score)]] = dict_high_lines[name_dict_high_lines[end_score.index(score)]]
print(len(dict_high_lines_temp))
dict_high_lines = dict_high_lines_temp.copy()

11


In [67]:
def dictW(alpha):
    # 运算构造参数W字典 
    dict_W = {}
    for elem in dict_high_lines:
        r = np.array(dict_high_lines[elem]["high_5"])
        W1= pow((1-r),2)
        W = np.exp(-W1/(2*pow(alpha,2)))
        dict_W[elem] = W
    return dict_W

def dictr(dict_W):
# 均方误差字典 （MSE=
    # r = max(|A×W - b|)
    dict_r = {}
    for elem,w in zip(dict_columns, dict_W):
        A = np.array(dict_columns[elem]["max_5x"].values) # x----->max_5x
        b = np.array(dict_columns[elem]["y"].values)
        W = dict_W[w]
        temp1 = np.dot(A,W) - b
        temp2 = np.square(temp1)
        temp3 = np.mean(temp2)
        dict_r[w] = temp3
    return dict_r

def dirlist(dict_r):
    #计算误差平方和    
    dir_list = []
    for elem in dict_r:
        dir_list.append(np.sqrt(dict_r[elem]))
    return dir_list

In [68]:
dev=10000000
lin=np.linspace(0.001,0.1,100)
for alpha in lin:
    
    dict_W = dictW(alpha)
    
    dict_r = dictr(dict_W)
 
    dir_list=dirlist(dict_r)

    temp=np.mean(dir_list)
                      
    #误差判断
    if(temp>=dev):
        continue
    else:
        dev=temp
        al=alpha

alpha=al
dict_W = dictW(alpha)
dict_r = dictr(dict_W)
dict_W

{'A2058_SKIN ': array([0.0062173 , 0.0080521 , 0.00874898, 0.00979676, 0.02190009,
        0.02973462, 0.04632054, 0.05774857, 0.06383277, 0.09706553,
        0.10701356, 0.13783963, 0.17870729]),
 'A2780_OVARY ': array([0.0038791 , 0.00670765, 0.00810559, 0.0090885 , 0.01656196,
        0.02262025, 0.03483852, 0.04076686, 0.04886619, 0.05086663,
        0.05774857, 0.17823187, 0.44106212]),
 'CAOV3_OVARY ': array([0.00168934, 0.00389108, 0.00456348, 0.00486469, 0.00986561,
        0.01677713, 0.02899853, 0.03281711, 0.04587567, 0.04632054,
        0.04886619, 0.11824038, 0.1283679 ]),
 'COLO320_LARGE_INTESTINE ': array([0.00107057, 0.00143674, 0.00377222, 0.00506881, 0.00618269,
        0.0080521 , 0.00986561, 0.01791231, 0.07075131, 0.07841707,
        0.08010039, 0.18549155, 0.44106212]),
 'HT29_LARGE_INTESTINE ': array([0.00389108, 0.00402252, 0.00597958, 0.00845222, 0.00852172,
        0.00979676, 0.01203467, 0.0196364 , 0.03607098, 0.04278827,
        0.13651805, 0.16044299, 0.36

In [69]:
# import pickle

# with open('细胞系.pkl', 'wb') as file:
#     pickle.dump(dict_W, file)
#     pickle.dump(dict_columns, file)
#     pickle.dump(dict_high_lines, file)

In [70]:
np.array(list(dict_r.values())).mean()

364.023623864947